In [18]:
import twint
import pandas as pd
import nest_asyncio
import time
nest_asyncio.apply()
pd.options.display.max_columns = 40


In [ ]:
# candidate = 'Biden'

# c = twint.Config()
# c.Search = candidate
# c.Lang = 'en'
# c.Since = '2020-10-01'
# c.Until = '2020-10-05'
# c.Store_csv = True
# c.Output = '../data/biden_tweets_all.csv'
# c.Hide_output = True
# twint.run.Search(c)

## Scrape Tweets

This is the workbook I used to create the below function.

Note...this takes a while. In order to prevent Twint's `client payload error`, I had to add randomized wait times between each Twitter query. I suggest running the below cells overnight to generate the files.

In [20]:
import datetime as dt
import time
import random

def arrange_dates(start_date, end_date):
    """Used in the get_election_tweets func. """
    
    dates = []
    delta = end_date - start_date
    
    for i in range(delta.days + 1):
        day = start_date + dt.timedelta(days=i)
        dates.append(day)
    
    return dates

def get_election_tweets(tweets_per_day=15000, start_date="2020-10-01", end_date="2020-11-03"):
    
    # get a list of dates
    f = "%Y-%m-%d"
    sd = dt.datetime.strptime(start_date, f)
    ed = dt.datetime.strptime(end_date, f)
    dates = arrange_dates(sd, ed)
    candidates = ['Biden', 'Trump']
    
    # fetch tweets & save CSVs (each CSV contains one day/candidate)
    
    lam_plus_1 = lambda x: (x + dt.timedelta(days=1)).strftime("%Y-%m-%d")
    wait_time = 50 # seconds; will use for sleep timer to avoid 'client payload' error
    print("""Waiting 60 seconds...\n""")
    time.sleep(60)
    for date in dates:
        for candidate in candidates:
            
            d = date.strftime("%Y-%m-%d")            
            print(f"""\n Begin scraping {candidate}'s tweets on {d}\n""")
            outfile = f"../data/tweets/{candidate}_{d}.csv"
            
            # twint search query details
            c = twint.Config()
            c.Search = candidate
            c.Lang = 'en'
            c.Since = d
            c.Until = lam_plus_1(date)
            c.Limit = tweets_per_day
            c.Store_csv = True
            c.Output = outfile
            c.Hide_output = True
            
            # run the search
            try:
                twint.run.Search(c)
                print(f"Scraping complete. File: {outfile} created.")
            except:
                time.sleep(90)
                continue
            
            # wait some time so script does not receive 'client payload' error
            print(f"Waiting 125-175 seconds...")
            time.sleep(wait_time + random.choice([70, 95, 120]) + (5 * random.uniform(0, 1)))
                       

In [ ]:
# get_election_tweets(start_date="2020-10-11", end_date="2020-11-02")

Alright! We now have most of our tweets, but the CSV's are incomplete on certain days. Below, I re-wrote the above function to only scrape specific days for a given candidate (to avoid re-scraping sections we've already completed successfully).

In [19]:
def get_candidate_tweets(dates, tweets_per_day, candidate='Trump'):
    
    f = "%Y-%m-%d"
    lam_plus_1 = lambda x: (x + dt.timedelta(days=1)).strftime("%Y-%m-%d")
    wait_time = 51.57848 # seconds; will use for sleep timer to avoid 'client payload' error
    print("""Waiting 60 seconds...\n""")
    time.sleep(60)
    
    for date in dates:
        d = date.strftime(f)            
        print(f"""\n Begin scraping {candidate}'s tweets on {d}\n""")
        outfile = f"../data/tweets/{candidate}_{d}.csv"

        # twint search query details
        c = twint.Config()
        c.Search = candidate
        c.Lang = 'en'
        c.Since = d
        c.Until = lam_plus_1(date)
        c.Limit = tweets_per_day
        c.Store_csv = True
        c.Output = outfile
        c.Hide_output = True

        # run the search
        try:
            twint.run.Search(c)
            print(f"Scraping complete. File: {outfile} created.")
        except:
            time.sleep(90)
            continue

        # wait some time so script does not receive 'client payload' error
        print(f"Waiting to avoid payload error...")
        time.sleep(wait_time + random.choice([100.129309, 135]) + (5 * random.uniform(0, 1)))

In [ ]:
trump_dates = ['2020-11-01',
               '2020-10-07',
               '2020-10-11', 
               '2020-10-14',
               '2020-10-17', 
               '2020-10-19',
               '2020-10-22',
               '2020-10-22',
               '2020-10-22',
               '2020-10-22',
               '2020-10-22']

get_candidate_tweets(trump_dates)



In [ ]:
biden_dates = ['2020-11-02']
time.sleep(120)
get_candidate_tweets(biden_dates, candidate='Biden')

In [ ]:
trump_dates = ['2020-10-11']
time.sleep(120)
get_candidate_tweets(trump_dates, candidate='Trump')

In [ ]:
trump_dates = ['2020-10-17']
time.sleep(120)
get_candidate_tweets(trump_dates, candidate='Trump')

In [24]:
def get_all_tweets_from_day(date, candidate='Trump', tweets_per_day=15000):
    
    f = "%Y-%m-%d"
    lam_plus_1 = lambda x: (x + dt.timedelta(days=1)).strftime("%Y-%m-%d")
    wait_time = 51.57848 # seconds; will use for sleep timer to avoid 'client payload' error
    
    
    d = dt.datetime.strptime(date, f)
    print(f"""\n Begin scraping {candidate}'s tweets on {d}\n""")
    outfile = f"../data/tweets/ALL_{candidate}_{date}.csv"

    # twint search query details
    c = twint.Config()
    c.Search = candidate
    c.Lang = 'en'
    c.Since = date
    c.Until = lam_plus_1(d)
    c.Store_csv = True
    c.Output = outfile
    c.Hide_output = True

    # run the search
    while True:
        try:
            twint.run.Search(c)
            print(f"Scraping complete. File: {outfile} created.")
        except:
            "Error caught. Restarting."
            time.sleep(90)
            continue

        # wait some time so script does not receive 'client payload' error
        print(f"Waiting to avoid payload error...")
        time.sleep(wait_time + random.choice([100.129309, 135.12478923749]) + (5 * random.uniform(0, 1)))

In [ ]:
get_all_tweets_from_day('2020-11-02', candidate='Trump')

In [ ]:
get_all_tweets_from_day('2020-11-02', candidate='Biden')


 Begin scraping Biden's tweets on 2020-11-02 00:00:00



[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Scraping complete. File: ../data/tweets/ALL_Biden_2020-11-02.csv created.
Waiting to avoid payload error...
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Scraping complete. File: ../data/tweets/ALL_Biden_2020-11-02.csv created.
Waiting to avoid payload error...


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Scraping complete. File: ../data/tweets/ALL_Biden_2020-11-02.csv created.
Waiting to avoid payload error...


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Scraping complete. File: ../data/tweets/ALL_Biden_2020-11-02.csv created.
Waiting to avoid payload error...
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Scraping complete. File: ../data/tweets/ALL_Biden_2020-11-02.csv created.
Waiting to avoid payload error...


In [ ]:
;d